<a href="https://colab.research.google.com/github/jh5723/SSE_Lab2/blob/main/video_pose_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies
Clone and import a stable version of the easy_ViTPose repository for a PyTorch implementation of the ViTPose model.

In [1]:
!git clone https://github.com/JunkyByte/easy_ViTPose.git
%cd easy_ViTPose
!git fetch --all
!git checkout 9ca694a982c1f6097e8ebf14ae7c12526da89ef0
!git log -1
!pip install -r requirements.txt
!pip install -e .
!pip install ipywidgets
import sys
sys.path.append('/content/easy_ViTPose')

Cloning into 'easy_ViTPose'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 924 (delta 244), reused 202 (delta 202), pack-reused 665 (from 1)
Receiving objects: 100% (924/924), 8.60 MiB | 5.81 MiB/s, done.
Resolving deltas: 100% (568/568), done.
/content/easy_ViTPose
Fetching origin
Note: switching to '9ca694a982c1f6097e8ebf14ae7c12526da89ef0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 9ca694a fix ins

Obtaining file:///content/easy_ViTPose
  Preparing metadata (setup.py) ... done
  Running setup.py develop for easy_ViTPose

^C


In [1]:
import os

from google.colab import drive
drive.mount('/content/my_drive') # force_remount=True if failing

Mounted at /content/my_drive


In [ ]:
%cd /content/my_drive/MyDrive/easy_ViTPose
!pip install -r requirements.txt
!pip install -e .
!pip install ipywidgets
import sys
sys.path.append('/content/my_drive/MyDrive/easy_ViTPose')

/content/my_drive/MyDrive/easy_ViTPose
Obtaining file:///content/my_drive/MyDrive/easy_ViTPose
  Preparing metadata (setup.py) ... done
  Running setup.py develop for easy_ViTPose


Once complete, restart session when prompted. If the prompt does not apepar, restart it manually (Runtime -> Restart session) to make sure installed package versions are updated locally.


# Download the object detection and pose estimation models

Mount Google Drive to access the pre-trained model weights. [Note: provide link to public weights folder here]

In [3]:
#@title Choose the specifications from the dropdown lists and run this cell

MODEL_SIZE = 's'  #@param ['s', 'b', 'l', 'h']
YOLO_SIZE = 'n'  #@param ['n', 's', 'm', 'l', 'x']
DATASET = 'aic'  #@param ['coco_25', 'coco', 'wholebody', 'mpii', 'aic', 'ap10k', 'apt36k']
ext = '.pth'
ext_yolo = '.pt'

root_path = '/content/my_drive/My Drive/tkd-ar/model_weights'
pose_path = os.path.join(root_path, 'ViTPose/pre-trained_weights')
pose_path = os.path.join(pose_path, f'{DATASET}/vitpose-' + MODEL_SIZE + f'-{DATASET}') + ext
detector_path = os.path.join(root_path, 'YOLOv8/pre-trained_weights')
detector_path = os.path.join(detector_path, 'yolov8' + YOLO_SIZE + ext_yolo)
print(f'Loading pre-trained pose model weights from: {pose_path}')
print(f'Loading pre-trained detector model weights from: {detector_path}')

Loading pre-trained pose model weights from: /content/my_drive/My Drive/tkd-ar/model_weights/ViTPose/pre-trained_weights/aic/vitpose-s-aic.pth
Loading pre-trained detector model weights from: /content/my_drive/My Drive/tkd-ar/model_weights/YOLOv8/pre-trained_weights/yolov8n.pt


Prepare the models for inference using the easy_ViTPose repository. Note, if an error occurrs at this step, restart the environment and rerun this cell to update the package versions.

Note setting is_video=True will load the basic SORT tracker to label individuals' bounding boxes between frames.

[Note provide options for better trackers later]

In [ ]:
# # Check on still image
# from easy_ViTPose import VitInference
# model = VitInference(pose_path, detector_path, MODEL_SIZE,
#                      dataset=DATASET, yolo_size=320, is_video=False)

# # Run inference on example image
# import numpy as np
# from io import BytesIO
# from PIL import Image
# from urllib.request import urlopen

# # Load image and run inference
# url = 'https://i.ibb.co/gVQpNqF/imggolf.jpg'
# img = np.array(Image.open(BytesIO(urlopen(url).read())), dtype=np.uint8)

# frame_keypoints = model.inference(img)
# img = model.draw(show_yolo=True)

# from google.colab.patches import cv2_imshow
# cv2_imshow(img[..., ::-1])

In [4]:
from easy_ViTPose import VitInference
model = VitInference(pose_path, detector_path, MODEL_SIZE,
                     dataset=DATASET, yolo_size=320, is_video=True)

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


# Load the videos for inference

Set paths for the unannotated video, the output video for the results of the inference, and the json file containing the results.

In [ ]:
# #@title Video paths

# import ipywidgets as widgets
# from IPython.display import display

# # Define the input widgets
# video_path_widget = widgets.Text(
#     value='/content/drive/MyDrive/Taekwondo videos/clipped_videos/P01/2_1_base_P01_base.mp4',
#     placeholder='Enter the path to the input video',
#     description='Video Path:',
#     disabled=False
# )

# output_video_path_widget = widgets.Text(
#     value='/content/drive/MyDrive/Taekwondo videos/clipped_videos/P01/output_2_1_base_P01_base.mp4',
#     placeholder='Enter the path to save the output video',
#     description='Output Video Path:',
#     disabled=False
# )

# output_json_path_widget = widgets.Text(
#     value='/content/drive/MyDrive/Taekwondo videos/clipped_videos/P01/2_1_base_P01_base.json',
#     placeholder='Enter the path to save the JSON output',
#     description='Output JSON Path:',
#     disabled=False
# )

# # Display the widgets
# display(video_path_widget)
# display(output_video_path_widget)
# display(output_json_path_widget)

# # Button to confirm the paths
# button = widgets.Button(description="Confirm")

# # Dictionary to store the paths
# paths = {}

# def on_button_click(b):
#     """
#     Captures the values entered by the user and stores them in the paths dictionary.
#     """
#     paths['video_path'] = video_path_widget.value
#     paths['output_video_path'] = output_video_path_widget.value
#     paths['output_json_path'] = output_json_path_widget.value

#     print(f"Video Path: {paths['video_path']}")
#     print(f"Output Video Path: {paths['output_video_path']}")
#     print(f"Output JSON Path: {paths['output_json_path']}")

# button.on_click(on_button_click)
# display(button)


Verify path selection:

In [ ]:
# video_path = paths.get('video_path', '')
# output_video_path = paths.get('output_video_path', '')
# output_json_path = paths.get('output_json_path', '')

# print(f"Using Video Path: {video_path}")
# print(f"Using Output Video Path: {output_video_path}")
# print(f"Using Output JSON Path: {output_json_path}")

In [5]:
#@title Choose the behaviour of the inference script

ANNOTATE_VIDEO = 'True'  #@param ['True', 'False']
SHOW_FRAMES = 'True'  #@param ['True', 'False']
OUTPUT_JSON_LABELS = 'True'  #@param ['True', 'False']
SHOW_BBOX = 'True'  #@param ['True', 'False']

prepare_output_video = ANNOTATE_VIDEO == 'True'
show_frames = SHOW_FRAMES == 'True'
output_json_labels = OUTPUT_JSON_LABELS == 'True'
show_bbox = SHOW_BBOX == 'True'

ANNOTATE_VIDEO=True will ensure that a copy of the input video with the bounding boxes and pose annotations  overlayed will be saved to the output_video_path.

SHOW_FRAMES=True displays the first frame of the video with the annotations overlayed to verify whether the model is working as expected.

OUTPUT_JSON_LABELS=True means that the frame-by-frame annotations will be written to the output_json_labels format in the structure laid out below.



Define the structure of the output JSON file.

In [6]:
import numpy as np
import json
from datetime import datetime

def create_json_output_format(
    keypoints_data, skeleton, width, height, fps, yolo_full_name, vitpose_full_name, dataset, interval_length=3.1
):
    """
    Create a JSON formatted dictionary with keypoints data and video properties, grouped into intervals.

    Args:
        keypoints_data (list): List of dictionaries containing keypoints for each frame.
        skeleton (list): List of skeleton keypoints.
        width (int): Width of the video frame.
        height (int): Height of the video frame.
        fps (float): Frames per second of the video.
        yolo_full_name (str): Full name of the YOLO model used.
        vitpose_full_name (str): Full name of the ViTPose model used.
        dataset (str): Dataset used for pose estimation.
        interval_length (float): Length of each interval in seconds (default is 3.1 seconds).

    Returns:
        dict: JSON formatted dictionary.
    """
    aspect_ratio = width / height  # Calculate aspect ratio

    # Calculate the exact number of frames per interval
    frames_per_interval_exact = fps * interval_length
    frames_per_interval_floor = int(np.floor(frames_per_interval_exact))
    remainder = frames_per_interval_exact - frames_per_interval_floor
    remainder_accumulated = 0

    # Skip the first two frames as warm-up
    keypoints_data = keypoints_data[2:]

    # Group frames into intervals
    techniques = {}
    technique_idx = 0

    start_idx = 0
    while start_idx < len(keypoints_data):
        # Determine the number of frames for the current interval
        interval_frames_count = frames_per_interval_floor
        remainder_accumulated += remainder
        if remainder_accumulated >= 1:
            interval_frames_count += 1
            remainder_accumulated -= 1

        # Extract frames for the current interval
        end_idx = min(start_idx + interval_frames_count, len(keypoints_data))
        interval_frames = keypoints_data[start_idx:end_idx]

        # Group frames under the current technique
        techniques[technique_idx] = {
            'frames': []
        }

        for frame_data in interval_frames:
            persons = []

            for person_id, keypoints_info in frame_data['keypoints'].items():
                keypoints_array = np.array(keypoints_info)
                num_keypoints = len(skeleton)  # Assume skeleton provides the number of keypoints

                # Check if the keypoints are already in the correct shape
                if keypoints_array.size != num_keypoints * 3:
                    print(f"Warning: Unexpected keypoints size for person {person_id} in frame {frame_data['frame']}")
                    continue

                # Use already-reshaped keypoints if possible
                keypoints_array = keypoints_array.reshape(num_keypoints, 3)
                keypoints_list = keypoints_array.tolist()

                persons.append({
                    'person_id': person_id,
                    'keypoints': keypoints_list
                })

            techniques[technique_idx]['frames'].append({
                'frame_number': frame_data['frame'],
                'persons': persons
            })

        # Check if the last technique needs padding
        if len(interval_frames) < interval_frames_count:
            last_frame = techniques[technique_idx]['frames'][-1]
            for _ in range(interval_frames_count - len(interval_frames)):
                techniques[technique_idx]['frames'].append(last_frame)

        # Increment technique index and update start index for the next interval
        technique_idx += 1
        start_idx = end_idx

    # Construct the final JSON structure
    return {
        'techniques': techniques,
        'detection_model': yolo_full_name,
        'pose_model': vitpose_full_name,
        'pose_format': dataset,
        'skeleton': skeleton,
        'camera_properties': {
            'width': width,
            'height': height,
            'aspect_ratio': aspect_ratio,
            'fps': fps
        },
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

Run inference on the chosen video. Note, the model must be reset each time to avoid confusion with the tracker module.

In [7]:
import numpy as np
import cv2
import json
import tqdm
import time
from datetime import datetime
from google.colab.patches import cv2_imshow
from easy_ViTPose.vit_utils.inference import NumpyEncoder, VideoReader
from easy_ViTPose.inference import VitInference
from easy_ViTPose.vit_utils.visualization import joints_dict

def setup_video_reader(video_path):
    """
    Initialize the video reader and capture video properties.

    Args:
        video_path (str): Path to the video file.

    Returns:
        VideoReader, int, int, int, float: Video reader object, width, height, total_frames, fps.
    """
    reader = VideoReader(video_path)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    return reader, width, height, total_frames, fps

def initialize_video_writer(output_video_path, width, height, fps):
    """
    Initialize the video writer for saving the output video.

    Args:
        output_video_path (str): Path to save the output video.
        width (int): Width of the video frames.
        height (int): Height of the video frames.
        fps (float): Frames per second of the video.

    Returns:
        cv2.VideoWriter: Video writer object.
    """
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    return out

def process_frames(reader, total_frames, model, num_keypoints):
    """
    Process each frame in the video, extract keypoints, and handle missing data.

    Args:
        reader (VideoReader): Video reader object.
        total_frames (int): Total number of frames in the video.
        model (VitInference): Pose estimation model.
        num_keypoints (int): Number of keypoints expected for each person.

    Returns:
        list: Processed keypoints data for all frames.
    """
    keypoints_data = []

    for ith, img in tqdm.tqdm(enumerate(reader), total=total_frames):
        t0 = time.time()

        if img is None:
            print(f"Frame {ith} is None, skipping.")
            continue

        # Perform inference to get keypoints
        frame_keypoints = model.inference(img)

        if not frame_keypoints:
            # Handle missing keypoints by labeling them as empty
            detected_keypoints = {
                # No person detected, use NaN for the person_id to preserve frame count
                "NaN": np.full((num_keypoints, 3), np.nan).flatten()
            }
        else:
            detected_keypoints = {}
            for person_id, keypoints in frame_keypoints.items():
                keypoints_array = np.array(keypoints).reshape(-1, 3)

                # Directly use detected keypoints, label missing keypoints as NaN
                detected_keypoints[person_id] = np.nan_to_num(keypoints_array.flatten(), nan=np.nan)

        keypoints_data.append({
            'frame': ith,
            'keypoints': detected_keypoints,
            'processing_time': time.time() - t0
        })

    return keypoints_data

def process_video_showing_output(reader, total_frames, model, num_keypoints, output_video_path, fps=30, frame_size=(640, 480)):
    """
    Process each frame in the video, extract keypoints, handle missing data, and optionally save output to a video file.

    Args:
        reader (VideoReader): Video reader object.
        total_frames (int): Total number of frames in the video.
        model (VitInference): Pose estimation model.
        num_keypoints (int): Number of keypoints expected for each person.
        output_video_path (str): Path to save the output video.
        save_video (bool): Flag to determine if the video should be saved.
        fps (int): Frames per second for the output video.
        frame_size (tuple): Dimensions of the video frame.

    Returns:
        list: Processed keypoints data for all frames.
    """
    keypoints_data = []

    out = initialize_video_writer(output_video_path, frame_size[0], frame_size[1], fps)

    for ith, img in tqdm.tqdm(enumerate(reader), total=total_frames):
        t0 = time.time()

        if img is None:
            print(f"Frame {ith} is None, skipping.")
            continue

        # Perform inference to get keypoints
        frame_keypoints = model.inference(img)

        img_with_keypoints = model.draw(show_yolo=True)[..., ::-1]  # Convert to BGR for OpenCV
        out.write(img_with_keypoints)

        if not frame_keypoints:
            # Handle missing keypoints by labeling them as empty
            detected_keypoints = {
                # No person detected, use NaN for the person_id
                "NaN": np.full((num_keypoints, 3), np.nan).flatten()
            }
        else:
            detected_keypoints = {}
            for person_id, keypoints in frame_keypoints.items():
                keypoints_array = np.array(keypoints).reshape(-1, 3)

                # Directly use detected keypoints, label missing keypoints as NaN
                detected_keypoints[person_id] = np.nan_to_num(keypoints_array.flatten(), nan=np.nan)

        keypoints_data.append({
            'frame': ith,
            'keypoints': detected_keypoints,
            'processing_time': time.time() - t0
        })

    out.release()
    return keypoints_data

def save_json_output(output_json_path, keypoints_data, model, width, height, fps, yolo_size, model_size, dataset):
    """
    Save the keypoints data to a JSON file.

    Args:
        output_json_path (str): Path to save the output JSON file.
        keypoints_data (list): Processed keypoints data.
        model (VitInference): Pose estimation model.
        width (int): Width of the video frames.
        height (int): Height of the video frames.
        fps (float): Frames per second of the video.
        yolo_size (str): YOLO model size.
        model_size (str): Pose model size.
        dataset (str): Dataset used for pose estimation.

    Returns:
        None
    """
    skeleton = joints_dict()[model.dataset]['keypoints']
    yolo_size_mapping = {
        's': 'yolov8s',
        'n': 'yolov8n',
        'm': 'yolov8m',
        'l': 'yolov8l',
        'x': 'yolov8x'
    }
    vitpose_size_mapping = {
        's': 'vitpose-s',
        'b': 'vitpose-b',
        'l': 'vitpose-l',
        'h': 'vitpose-h'
    }
    yolo_full_name = yolo_size_mapping.get(yolo_size, 'unknown_yolo_size')
    vitpose_full_name = f"{vitpose_size_mapping.get(model_size, 'unknown_vitpose_size')}-{dataset}"

    output_data = create_json_output_format(keypoints_data, skeleton, width, height, fps, yolo_full_name, vitpose_full_name, dataset)

    with open(output_json_path, 'w') as f:
        json.dump(output_data, f, cls=NumpyEncoder)

def display_first_frame(output_video_path):
    """
    Display the first frame of the processed video for demonstration.

    Args:
        output_video_path (str): Path to the output video file.

    Returns:
        None
    """
    cap = cv2.VideoCapture(output_video_path)
    ret, frame = cap.read()
    if ret:
        cv2_imshow(frame)
    cap.release()

def output_statistics(keypoints_data, fps_list, tot_time):
    """
    Output performance statistics of the video processing.

    Args:
        keypoints_data (list): Processed keypoints data.
        fps_list (list): List of frame processing times.
        tot_time (float): Total processing time.

    Returns:
        None
    """
    tot_poses = sum(len(k['keypoints']) for k in keypoints_data)
    print(f'Mean inference FPS: {1 / np.mean(fps_list):.2f}')
    print(f'Total poses predicted: {tot_poses}, mean per frame: {(tot_poses / len(keypoints_data)):.2f}')
    print(f'Mean FPS per pose: {(tot_poses / tot_time):.2f}')

def process_video(model, keypoints, video_path, output_json_path, output_video_path='output_video.mp4', prepare_output_video=False, show_frames=False, yolo_size='s', model_size='b', dataset='coco', interpolate_threshold=3):
    """
    Main function to process a video file, extract and smooth pose keypoints, and save results.

    Args:
        video_path (str): Path to the input video file.
        output_json_path (str): Path to save the output JSON file.
        output_video_path (str): Path to save the output video with keypoints overlay.
        prepare_output_video (bool): Whether to prepare an output video with keypoints overlay.
        show_frames (bool): Whether to display the first frame of the processed video.
        yolo_size (str): Size of the YOLO model.
        model_size (str): Size of the pose estimation model.
        dataset (str): Dataset format for pose estimation.
        interpolate_threshold (int): Maximum number of consecutive missing frames to interpolate.

    Returns:
        None
    """
    reader, width, height, total_frames, fps = setup_video_reader(video_path)

    if prepare_output_video:
        print("Preparing output video...")
        keypoints_data = process_video_showing_output(reader, total_frames, model, keypoints, output_video_path, fps=fps, frame_size=(width, height))
    else:
        print("Processing frames without output video...")
        keypoints_data = process_frames(reader, total_frames, model, keypoints)

    save_json_output(output_json_path, keypoints_data, model, width, height, fps, yolo_size, model_size, dataset)

    if show_frames:
        display_first_frame(output_video_path)

    fps_list = [frame['processing_time'] for frame in keypoints_data]
    tot_time = sum(fps_list)
    output_statistics(keypoints_data, fps_list, tot_time)



In [8]:
def get_keypoint_no(skel_format):
  if skel_format == 'coco':
    return 17
  elif skel_format == 'coco_25':
    return 25
  elif skel_format == 'mpii':
    return 16
  elif skel_format == 'aic':
    return 14
  elif skel_format == 'apt36k':
    print("Warning: This pose format is for labelling animals.")
    return 36
  elif skel_format == 'ap10k':
    print("Warning: This pose format is for labelling animals.")
    return 10 # Animal pose estimation
  elif skel_format == 'wholebody':
    print("Warning: Labels 132 keypoints, most of which are in the face and hands.")
    return 132 # Whole body COCO pose estimation, with many face and hand keyoints
  else:
    raise ValueError(f"Unsupported skeleton format: {skel_format}")

Test annotations

In [ ]:
# Test annotations
keypoints = get_keypoint_no(DATASET)
participant = 'P02'
camera = 'c2'
input_path = f'/content/my_drive/MyDrive/tkd-ar/video_data/clipped_videos/{participant}/{participant}_{camera}.mp4'
output_path = f'/content/my_drive/MyDrive/test_length_{participant}_{camera}.mp4'
json_path = f'/content/my_drive/MyDrive/test_length_{participant}_{camera}.json'
# input_path = '/content/my_drive/MyDrive/Taekwondo videos/source_files/P08_c1_clipped.mp4'
# json_path = '/content/my_drive/MyDrive/Taekwondo videos/source_files/P08_c1_clipped_output.json'
# output_path = '/content/my_drive/MyDrive/Taekwondo videos/source_files/P08_c1_clipped_output.mp4'
process_video(model, keypoints, input_path, json_path, output_path, prepare_output_video=prepare_output_video, show_frames=show_frames, yolo_size=YOLO_SIZE, model_size=MODEL_SIZE, dataset=DATASET)

Preparing output video...


  5%|▌         | 437/8479 [00:31<08:31, 15.72it/s]

In [ ]:
import os

def generate_video_paths(root, vitpose_version, yolo_version, dataset):
    """
    Generate file paths for video processing for all participants and sessions.

    Args:
        root (str): Root directory where participant folders are located.
        vitpose_version (str): Version of the ViTPose model used for naming outputs.
        yolo_version (str): Version of the YOLO model used for naming outputs.
        dataset (str): Dataset name used for naming the subfolder.

    Returns:
        list: List of lists containing paths [input_video, output_video, output_json].
    """
    video_paths = []

    # Define the base output directory structure
    base_output_dir = os.path.join(root, f'{dataset}_vitpose_{vitpose_version}_yolo_{yolo_version}')

    # Create base output directory if it does not exist
    if not os.path.exists(base_output_dir):
        os.makedirs(base_output_dir)
        print(f"Created directory: {base_output_dir}")

    # Traverse the root directory to find participant folders
    for participant_folder in os.listdir(root):
        participant_path = os.path.join(root, participant_folder)
        print(f"Checking folder: {participant_path}")

        # Ensure it's a directory and matches the "PXX" format
        if os.path.isdir(participant_path) and participant_folder.startswith('P'):
            print(f"Processing participant folder: {participant_folder}")
            # Traverse each file in the participant folder
            for file in os.listdir(participant_path):
                print(f"Found file: {file}")
                if file.endswith('.mp4'):  # Look for MP4 video files
                    print(f"Processing video file: {file}")
                    # Construct input and output paths
                    input_video = os.path.join(participant_path, file)
                    session_name = os.path.splitext(file)[0]  # Remove the file extension for naming

                    output_video = f'output_{session_name}_vitpose_{vitpose_version}_yolo_{yolo_version}.mp4'
                    output_json = f'{session_name}_vitpose_{vitpose_version}_yolo_{yolo_version}.json'

                    # Define the full path for JSON output
                    output_video_path = os.path.join(base_output_dir, participant_folder, output_video)
                    output_json_path = os.path.join(base_output_dir, participant_folder, output_json)

                    # Create participant subdirectory if it does not exist
                    participant_output_dir = os.path.join(base_output_dir, participant_folder)
                    if not os.path.exists(participant_output_dir):
                        os.makedirs(participant_output_dir)
                        print(f"Created directory: {participant_output_dir}")

                    # Check if output JSON already exists in the JSON directory
                    if os.path.exists(output_json_path):
                        print(f"Skipping {input_video} because JSON output already exists as {output_json}")
                        continue

                    # Append paths to video_paths list
                    video_paths.append([
                        input_video,
                        output_video_path,
                        output_json_path
                    ])

    return video_paths

MODEL_SIZE = 's'  #@param ['s', 'b', 'l', 'h']
YOLO_SIZE = 'n'  #@param ['n', 's', 'm', 'l', 'x']
DATASET = 'aic'  #@param ['coco_25', 'coco', 'wholebody', 'mpii', 'aic', 'ap10k', 'apt36k']
ext = '.pth'
ext_yolo = '.pt'

root_path = '/content/my_drive/My Drive/pose_models'
pose_path = os.path.join(root_path, 'ViTPose/pre-trained_weights')
pose_path = os.path.join(pose_path, f'{DATASET}/vitpose-' + MODEL_SIZE + f'-{DATASET}') + ext
detector_path = os.path.join(root_path, 'YOLOv8/pre-trained_weights')
detector_path = os.path.join(detector_path, 'yolov8' + YOLO_SIZE + ext_yolo)
print(f'Loading pre-trained pose model weights from: {pose_path}')
print(f'Loading pre-trained detector model weights from: {detector_path}')
ANNOTATE_VIDEO = 'False'  #@param ['True', 'False']
SHOW_FRAMES = 'False'  #@param ['True', 'False']
OUTPUT_JSON_LABELS = 'True'  #@param ['True', 'False']
SHOW_BBOX = 'False'  #@param ['True', 'False']

prepare_output_video = ANNOTATE_VIDEO == 'True'
show_frames = SHOW_FRAMES == 'True'
output_json_labels = OUTPUT_JSON_LABELS == 'True'
show_bbox = SHOW_BBOX == 'True'

from easy_ViTPose import VitInference
model = VitInference(pose_path, detector_path, MODEL_SIZE,
                     dataset=DATASET, yolo_size=320, is_video=True)

# Generate video paths for all videos in the directory
video_paths = generate_video_paths('/content/my_drive/MyDrive/Taekwondo videos/clipped_videos', MODEL_SIZE, YOLO_SIZE, DATASET)

keypoints = get_keypoint_no(DATASET)

# Process each video using the generated paths
for video in video_paths:
    input_video, output_video, output_json = video

    print(f"Processing {input_video}:")
    process_video(model, keypoints, input_video, output_json, output_video, yolo_size=YOLO_SIZE, model_size=MODEL_SIZE, dataset=DATASET)


Loading pre-trained pose model weights from: /content/my_drive/My Drive/pose_models/ViTPose/pre-trained_weights/coco_25/vitpose-h-coco_25.pth
Loading pre-trained detector model weights from: /content/my_drive/My Drive/pose_models/YOLOv8/pre-trained_weights/yolov8m.pt
Checking folder: /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P01
Processing participant folder: P01
Found file: P01_c1.mp4
Processing video file: P01_c1.mp4
Skipping /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P01/P01_c1.mp4 because JSON output already exists as P01_c1_vitpose_h_yolo_m.json
Found file: P01_c2.mp4
Processing video file: P01_c2.mp4
Skipping /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P01/P01_c2.mp4 because JSON output already exists as P01_c2_vitpose_h_yolo_m.json
Found file: P01_c3.mp4
Processing video file: P01_c3.mp4
Skipping /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P01/P01_c3.mp4 because JSON output already exists as P01_c3_vitpose_h_yolo_m.json


100%|█████████▉| 39061/39111 [2:44:16<00:12,  3.96it/s]


Mean inference FPS: 4.15
Total poses predicted: 88792, mean per frame: 2.27
Mean FPS per pose: 9.43
Processing /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P08/P08_c4.mp4:
Processing frames without output video...


100%|█████████▉| 39102/39249 [3:07:00<00:42,  3.48it/s]


Mean inference FPS: 3.52
Total poses predicted: 105330, mean per frame: 2.69
Mean FPS per pose: 9.49
Processing /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P08/P08_c5b.mp4:
Processing frames without output video...


...

Mean inference FPS: 3.99
Total poses predicted: 5060, mean per frame: 2.34
Mean FPS per pose: 9.34
Processing /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P08/P08_c6.mp4:
Processing frames without output video...


100%|█████████▉| 38844/38845 [3:21:17<00:00,  3.22it/s]


Mean inference FPS: 3.34
Total poses predicted: 109863, mean per frame: 2.83
Mean FPS per pose: 9.44
Processing /content/my_drive/MyDrive/Taekwondo videos/clipped_videos/P08/P08_c5a.mp4:
Processing frames without output video...


100%|█████████▉| 32292/32305 [1:32:02<00:02,  5.85it/s]


Mean inference FPS: 6.03
Total poses predicted: 49389, mean per frame: 1.53
Mean FPS per pose: 9.23


Single file run:

In [ ]:
# import numpy as np
# import cv2
# from google.colab.patches import cv2_imshow
# import json
# from easy_ViTPose.vit_utils.inference import NumpyEncoder, VideoReader
# from easy_ViTPose.inference import VitInference
# from easy_ViTPose.vit_utils.visualization import joints_dict
# import tqdm
# import time
# from datetime import datetime
# from scipy.signal import savgol_filter  # Savitzky-Golay filter for smoothing


# model.reset()

# reader = VideoReader(video_path)

# cap = cv2.VideoCapture(video_path)
# fps = cap.get(cv2.CAP_PROP_FPS)
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# cap.release()

# if prepare_output_video:
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
#     out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# # Process the video frame by frame
# keypoints_data = []
# fps_list = []
# tot_time = 0.0
# print("Total frames in video:", total_frames)

# for (ith, img) in tqdm.tqdm(enumerate(reader), total=total_frames):
#     t0 = time.time()

#     if img is None:
#         print(f"Frame {ith} is None, skipping.")
#         continue

#     frame_keypoints = model.inference(img)
#     keypoints_data.append({'frame': ith, 'keypoints': frame_keypoints})

#     delta = time.time() - t0
#     tot_time += delta
#     fps_list.append(delta)

#     if prepare_output_video:
#         img_with_keypoints = model.draw(show_yolo=True)[..., ::-1]  # Convert to BGR for OpenCV
#         out.write(img_with_keypoints)

# if prepare_output_video:
#     out.release()
# cv2.destroyAllWindows()

# # Display the first frame of the processed video as a demonstration
# if SHOW_FRAMES:
#     cap = cv2.VideoCapture(output_video_path)
#     ret, frame = cap.read()
#     if ret:
#         cv2_imshow(frame)
#     cap.release()

# # Save the keypoints to a JSON file if required
# if output_json_labels:
#     skeleton = joints_dict()[model.dataset]['keypoints']
#     output_data = create_json_output_format(keypoints_data, skeleton, width, height, fps)

#     with open(output_json_path, 'w') as f:
#         json.dump(output_data, f, cls=NumpyEncoder)

# # Output performance statistics
# tot_poses = sum(len(k['keypoints']) for k in keypoints_data)
# print(f'Mean inference FPS: {1 / np.mean(fps_list):.2f}')
# print(f'Total poses predicted: {tot_poses}, mean per frame: {(tot_poses / len(keypoints_data)):.2f}')
# print(f'Mean FPS per pose: {(tot_poses / tot_time):.2f}')

Quick single file run (no output video)

In [ ]:
# import numpy as np
# import cv2
# import json
# from easy_ViTPose.vit_utils.inference import NumpyEncoder, VideoReader
# from easy_ViTPose.inference import VitInference
# import tqdm
# import time

# # Initialize the video reader
# reader = VideoReader(video_path)

# # Get video properties
# cap = cv2.VideoCapture(video_path)
# fps = cap.get(cv2.CAP_PROP_FPS)
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# cap.release()

# # Process the video frame by frame
# fps_list = []
# tot_time = 0.0
# print("Total frames in video:", total_frames)

# for ith, img in tqdm.tqdm(enumerate(reader), total=total_frames):
#     t0 = time.time()

#     if img is None:
#         print(f"Frame {ith} is None, skipping.")
#         continue

#     # Perform inference
#     model.inference(img)

#     delta = time.time() - t0
#     tot_time += delta
#     fps_list.append(delta)

# # Output performance statistics
# print(f'\nMean inference FPS: {1 / np.mean(fps_list):.2f}')
# print(f'Total inference time: {tot_time:.2f} seconds')
# print(f'Total poses predicted: {tot_poses}, mean per frame: {(tot_poses / len(keypoints_data)):.2f}')
# print(f'Total frames processed: {len(fps_list)}')
# print(f'Mean FPS per pose: {(tot_poses / tot_time):.2f}')